In [1]:
import hydra
import torch
from tqdm.notebook import tqdm
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from collections import defaultdict
import pandas as pd


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def eval_iterations(optimizer, datamodule, N: int = 3):
    optimizer.max_iters = N
    optimizer.max_optims = 1
    time_tracker = TimeTracker()
    # initial evaluation no optimization
    p_loss = []
    v_loss = []
    g_loss = []
    for batch in datamodule.val_dataloader():
        with torch.no_grad():
            batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
            time_tracker.start("optimize")
            out = optimizer(batch)
            time_tracker.stop("optimize")
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            p_loss.append(loss_info["loss_param"])
            v_loss.append(loss_info["loss_vertices"])
            g_loss.append(loss_info["loss_geometric"])
    iters_p_loss = torch.stack(p_loss).mean().item()
    iters_g_loss = torch.stack(g_loss).mean().item()
    iters_v_loss = torch.stack(v_loss).mean().item()
    t_perf = [torch.tensor(t.time_ms) for t in list(time_tracker.tracks.values())[0]]
    iters_time = torch.stack(t_perf).median().item()
    return iters_p_loss, iters_g_loss, iters_v_loss, iters_time


def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=kinect"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path):
    cfg = load_config("train", ["data=kinect"])
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


def load_icp_optimizer(flame, renderer, overrides):
    o = ["data=kinect", "optimizer.output_dir=none"] + overrides
    cfg = load_config("train", o)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    optimizer = hydra.utils.instantiate(cfg.optimizer)
    regularize = hydra.utils.instantiate(cfg.regularize)
    icp_optimizer = hydra.utils.instantiate(
        cfg.framework,
        flame=flame,
        logger=None,
        renderer=renderer,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        optimizer=optimizer,
        weighting=weighting,
    )
    return icp_optimizer.to("cuda")


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame, jump_size):
    cfg = load_config("train", ["data=kinect"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
            jump_size=jump_size,
        ),
    )
    datamodule.setup("fit")
    return datamodule

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
N = 1
start_frame = 48 
end_frame = 56
# step_size = 0.7

# loadings
times = defaultdict(dict)
p_losses = defaultdict(dict)
g_losses = defaultdict(dict)
v_losses = defaultdict(dict)
flame, renderer = load_flame_renderer()

for jump_size in tqdm([1, 2, 4, 8]):
    datamodule = load_datamodule(renderer, start_frame, end_frame, jump_size)

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.0
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-00"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.1 
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-01"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.3 
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-03"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.5
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-05"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.7
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-07"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.9
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-09"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 1.0
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-10"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

Creating GL context for cuda device 0
Successfully initialized EGL version 1.5
Successfully initialized OpenGL version 4.6.0 NVIDIA 535.183.01


  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
desired_order = ["icp-00", "icp-01", "icp-03", "icp-05", "icp-07", "icp-09", "icp-10"]

# Create the DataFrame for p_losses
p_losses_df = pd.DataFrame(p_losses).transpose()
p_losses_df.columns = [f"0->{c}" for c in p_losses_df.columns]
p_losses_df = p_losses_df.reindex(desired_order)
p_losses_df.columns = pd.MultiIndex.from_tuples([("FLAME (norm)", c) for c in p_losses_df.columns])

# Create the DataFrame for v_losses
v_losses_df = pd.DataFrame(v_losses).transpose()
v_losses_df.columns = [f"0->{c}" for c in v_losses_df.columns]
v_losses_df = v_losses_df.reindex(desired_order)
v_losses_df.columns = pd.MultiIndex.from_tuples([("Vertices (mm)", c) for c in v_losses_df.columns])

# Concatenate the two DataFrames
pd.concat([p_losses_df, v_losses_df] , axis=1)

FLAME (norm)                               Vertices (mm)            \
               0->1      0->2      0->4      0->8          0->1      0->2   
icp-00     0.323893  0.500221  0.599251  0.588814      6.464082  7.183959   
icp-01     0.304442  0.468137  0.560966  0.554277      6.383245  7.007974   
icp-03     0.305419  0.441358  0.526187  0.529414      6.244434  6.687375   
icp-05     0.355931  0.466641  0.554298  0.560159      6.144586  6.423134   
icp-07     0.436493  0.536660  0.632270  0.646035      6.094942  6.243714   
icp-09     0.533971  0.636797  0.744534  0.767452      6.102095  6.190627   
icp-10     0.586744  0.694349  0.810962  0.835937      6.125517  6.216054   

                            
            0->4      0->8  
icp-00  8.352511  9.702329  
icp-01  8.066373  9.511890  
icp-03  7.532995  9.181485  
icp-05  7.072595  8.933166  
icp-07  6.723217  8.802304  
icp-09  6.547132  8.848352  
icp-10  6.544899  8.945674